In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
import os
from utils.common import (
    preprocess_mask2former,
    m2f_extract_pred_maps_and_masks,
    mask2former_auto_image_processor,
)
from utils.dataset_utils import (
    get_cadisv2_dataset,
    get_cataract1k_dataset,
    ZEISS_CATEGORIES,
)
from transformers import Mask2FormerForUniversalSegmentation
from torch.utils.data import DataLoader
import evaluate
import torch.optim as optim
from tqdm import tqdm
from torch.utils.tensorboard import SummaryWriter
import numpy as np

/notebooks/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/notebooks/venv/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [3]:
NUM_CLASSES = len(ZEISS_CATEGORIES) - 3 + 1 # Remove class incremental, add background

# Load model
model = Mask2FormerForUniversalSegmentation.from_pretrained(
    "facebook/mask2former-swin-large-ade-semantic",
    num_labels=NUM_CLASSES - 1,
    ignore_mismatched_sizes=True,
)

/notebooks/venv/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of Mask2FormerForUniversalSegmentation were not initialized from the model checkpoint at facebook/mask2former-swin-large-ade-semantic and are newly initialized because the shapes did not match:
- class_predictor.bias: found shape torch.Size([151]) in the checkpoint and torch.Size([12]) in the model instantiated
- class_predictor.weight: found shape torch.Size([151, 256]) in the checkpoint and torch.Size([12, 256]) in the model instantiated
- criterion.empty_weight: found shape torch.Size([151]) in the checkpoint and torch.Size([12]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [18]:
# Domain incremental datasets
train_cadis_ds, val_cadis_ds, test_cadis_ds = get_cadisv2_dataset(
    "../data/CaDISv2", domain_incremental=True
)
train_cataract_ds, val_cataract_ds, test_cataract_ds = get_cataract1k_dataset(
    "../data/cataract-1k", domain_incremental=True
)

# First case: CADIS + CATARACT at the same time
merged_train = torch.utils.data.ConcatDataset([train_cataract_ds, train_cadis_ds])
merged_val = torch.utils.data.ConcatDataset([val_cataract_ds, val_cadis_ds])

# Define dataloader params
N_WORKERS = 8
BATCH_SIZE = 16
SHUFFLE = True
DROP_LAST = True

# Define dataloader
train_merged_loader = DataLoader(
    train_cataract_ds,#train_cadis_ds,
    batch_size=BATCH_SIZE,
    shuffle=SHUFFLE,
    num_workers=N_WORKERS,
    drop_last=DROP_LAST,
    pin_memory=True,
    collate_fn=preprocess_mask2former,
)
val_merged_loader = DataLoader(
    val_cataract_ds,#merged_val,
    batch_size=BATCH_SIZE,
    shuffle=SHUFFLE,
    num_workers=N_WORKERS,
    drop_last=DROP_LAST,
    pin_memory=True,
    collate_fn=preprocess_mask2former,
)

In [19]:
# Check if CUDA is available, otherwise use CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(f"Using device: {device}")

Using device: cuda


In [20]:
# Training
NUM_EPOCHS = 50
LEARNING_RATE = 1e-4
LR_MULTIPLIER = 0.1
BACKBONE_LR = LEARNING_RATE * LR_MULTIPLIER
WEIGHT_DECAY = 0.5
# dice = Dice(average='micro')

# lambda_CE=5.0
# lambda_dice=5.0
metric = evaluate.load("mean_iou")
encoder_params = [
    param
    for name, param in model.named_parameters()
    if name.startswith("model.pixel_level_module.encoder")
]
decoder_params = [
    param
    for name, param in model.named_parameters()
    if name.startswith("model.pixel_level_module.decoder")
]
transformer_params = [
    param
    for name, param in model.named_parameters()
    if name.startswith("model.transformer_module")
]
optimizer = optim.AdamW(
    [
        {"params": encoder_params, "lr": BACKBONE_LR},
        {"params": decoder_params},
        {"params": transformer_params},
    ],
    lr=LEARNING_RATE,
    weight_decay=WEIGHT_DECAY,
)

scheduler = optim.lr_scheduler.PolynomialLR(
    optimizer, total_iters=NUM_EPOCHS, power=0.9
)

In [21]:
# Tensorboard setup
out_dir="outputs/"
if not os.path.exists(out_dir):
    os.makedirs(out_dir)
if not os.path.exists(out_dir+"runs"):
    os.makedirs(out_dir+"runs")
%load_ext tensorboard
%tensorboard --logdir outputs/runs

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 1384), started 0:19:45 ago. (Use '!kill 1384' to kill it.)

In [22]:
# Tensorboard logging
writer = SummaryWriter(log_dir=out_dir + "runs")

# Model checkpointing
model_name = "m2f_swin_ade20_merged_datasets"
model_dir = out_dir + "models/"
if not os.path.exists(model_dir):
    print("Store weights in: ", model_dir)
    os.makedirs(model_dir)

best_model_dir = model_dir + f"{model_name}/best_models/"
if not os.path.exists(model_dir):
    print("Store best model weights in: ", best_model_dir)
    os.makedirs(best_model_dir)
final_model_dir = model_dir + f"{model_name}/final_model/"
if not os.path.exists(model_dir):
    print("Store final model weights in: ", final_model_dir)
    os.makedirs(final_model_dir)

Store weights in:  outputs/models/


In [11]:
!rm -r outputs

In [9]:
!CUDA_LAUNCH_BLOCKING=1

In [23]:
# For storing the model
best_val_metric = -np.inf

# Move model to device
model.to(device)

for epoch in range(NUM_EPOCHS):
    model.train()
    train_running_loss = 0.0
    val_running_loss = 0.0

    # Set up tqdm for the training loop
    train_loader = tqdm(
        train_merged_loader, desc=f"Epoch {epoch + 1}/{NUM_EPOCHS} Training"
    )
    
    for batch in train_loader:
        # Move everything to the device
        batch["pixel_values"] = batch["pixel_values"].to(device)
        batch["pixel_mask"] = batch["pixel_mask"].to(device)
        batch["mask_labels"] = [entry.to(device) for entry in batch["mask_labels"]]
        batch["class_labels"] = [entry.to(device) for entry in batch["class_labels"]]
        
        
        # Compute output and loss
        outputs = model(**batch)
        
        loss = outputs.loss
        
        
        # Compute gradient and perform step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Record losses
        current_loss = loss.item() * batch["pixel_values"].size(0)
        train_running_loss += current_loss
        train_loader.set_postfix(loss=f"{current_loss:.4f}")

        # Extract and compute metrics
        pred_maps, masks = m2f_extract_pred_maps_and_masks(
            batch, outputs, mask2former_auto_image_processor
        )
        metric.add_batch(references=masks, predictions=pred_maps)
        
    # After compute the batches that were added are deleted
    mean_train_iou = metric.compute(
        num_labels=NUM_CLASSES, ignore_index=255, reduce_labels=True
    )["mean_iou"]

    # Validation phase
    model.eval()
    val_loader = tqdm(
        val_merged_loader, desc=f"Epoch {epoch + 1}/{NUM_EPOCHS} Validation"
    )
    with torch.no_grad():
        for batch in val_loader:
            # Move everything to the device
            batch["pixel_values"] = batch["pixel_values"].to(device)
            batch["pixel_mask"] = batch["pixel_mask"].to(device)
            batch["mask_labels"] = [entry.to(device) for entry in batch["mask_labels"]]
            batch["class_labels"] = [
                entry.to(device) for entry in batch["class_labels"]
            ]
            # Compute output and loss
            outputs = model(**batch)
            
            
            loss = outputs.loss
            # Record losses
            current_loss = loss.item() * batch["pixel_values"].size(0)
            val_running_loss += current_loss
            val_loader.set_postfix(loss=f"{current_loss:.4f}")

            # Extract and compute metrics
            pred_maps, masks = m2f_extract_pred_maps_and_masks(
                batch, outputs, mask2former_auto_image_processor
            )
            metric.add_batch(references=masks, predictions=pred_maps)
            
    
    # After compute the batches that were added are deleted
    mean_val_iou = metric.compute(
        num_labels=NUM_CLASSES, ignore_index=255, reduce_labels=True
    )["mean_iou"]

    epoch_train_loss = train_running_loss / len(train_merged_loader.dataset)
    epoch_val_loss = val_running_loss / len(val_merged_loader.dataset)

    writer.add_scalar(f"Loss/train_{model_name}", epoch_train_loss, epoch + 1)
    writer.add_scalar(f"Loss/val_{model_name}", epoch_val_loss, epoch + 1)
    writer.add_scalar(f"mIoU/train_{model_name}", mean_train_iou, epoch + 1)
    writer.add_scalar(f"mIoU/val_{model_name}", mean_val_iou, epoch + 1)

    if mean_val_iou > best_val_metric:
        best_val_metric = mean_val_iou
        model.save_pretrained(best_model_dir)

    tqdm.write(
        f"Epoch {epoch + 1}/{NUM_EPOCHS}, Train Loss: {epoch_train_loss:.4f}, Train mIoU: {mean_train_iou:.4f}, Validation Loss: {epoch_val_loss:.4f}, Validation mIoU: {mean_val_iou:.4f}"
    )
    

Epoch 1/50 Training:   0%|          | 0/451 [00:03<?, ?it/s, loss=349.5780]
/root/.cache/huggingface/modules/evaluate_modules/metrics/evaluate-metric--mean_iou/9e450724f21f05592bfb0255fe2fa576df8171fa060d11121d8aecfff0db80d0/mean_iou.py:259: RuntimeWarning: invalid value encountered in divide
  iou = total_area_intersect / total_area_union
/root/.cache/huggingface/modules/evaluate_modules/metrics/evaluate-metric--mean_iou/9e450724f21f05592bfb0255fe2fa576df8171fa060d11121d8aecfff0db80d0/mean_iou.py:260: RuntimeWarning: invalid value encountered in divide
  acc = total_area_intersect / total_area_label
Epoch 1/50 Validation:   0%|          | 0/56 [00:03<?, ?it/s, loss=286.0847]


In [ ]:
"""
TODO:
  1. Write the test set evaluation and export metrics.
  2. The preprocessor reduces the size of all images AND masks, so that we can train them.
    This raises the question if such a training procedure is good enough. We need to try and
    train a model, then postprocess the predicted maps the original image size and validate
    the data visually.

  3. We currently load a Mask2Former model that is pretrained using the swin backbone on
    ADE. Can we reinitialize the other weights s.t. we end up having a pretrained backbone
    and untrained pixel decoder and transformer decoder.

    => Mask2Former config with pretrained backbone and newly initialized model.

  4. Consider Weights and Biases. I've heard only good things about it and maybe we can give it a try.
"""